In [8]:
import pandas as pd

from pokemon_tcg_service import PokemonTCGServer

server = PokemonTCGServer()
cards = server.get_cards(page_size=10, order_by="set.releaseDate", query="set.name:generations")

print(cards)

{'data': [{'id': 'g1-1', 'name': 'Venusaur-EX', 'supertype': 'Pokémon', 'subtypes': ['Basic', 'EX'], 'hp': '180', 'types': ['Grass'], 'evolvesTo': ['M Venusaur-EX'], 'rules': ['Pokémon-EX rule: When a Pokémon-EX has been Knocked Out, your opponent takes 2 Prize cards.'], 'attacks': [{'name': 'Frog Hop', 'cost': ['Grass', 'Colorless', 'Colorless'], 'convertedEnergyCost': 3, 'damage': '40+', 'text': 'Flip a coin. If heads, this attack does 40 more damage.'}, {'name': 'Poison Impact', 'cost': ['Grass', 'Grass', 'Colorless', 'Colorless'], 'convertedEnergyCost': 4, 'damage': '80', 'text': "Your opponent's Active Pokémon is now Asleep and Poisoned."}], 'weaknesses': [{'type': 'Fire', 'value': '×2'}], 'retreatCost': ['Colorless', 'Colorless', 'Colorless', 'Colorless'], 'convertedRetreatCost': 4, 'set': {'id': 'g1', 'name': 'Generations', 'series': 'XY', 'printedTotal': 83, 'total': 117, 'legalities': {'unlimited': 'Legal', 'expanded': 'Legal'}, 'ptcgoCode': 'GEN', 'releaseDate': '2016/02/22',

In [ ]:
df = pd.DataFrame(cards)
print(df)
print(df.shape)

                                                data  page  pageSize  count  \
0  {'id': 'g1-1', 'name': 'Venusaur-EX', 'superty...     1        10     10   
1  {'id': 'g1-2', 'name': 'M Venusaur-EX', 'super...     1        10     10   
2  {'id': 'g1-3', 'name': 'Caterpie', 'supertype'...     1        10     10   
3  {'id': 'g1-4', 'name': 'Metapod', 'supertype':...     1        10     10   
4  {'id': 'g1-5', 'name': 'Butterfree', 'supertyp...     1        10     10   
5  {'id': 'g1-6', 'name': 'Paras', 'supertype': '...     1        10     10   
6  {'id': 'g1-7', 'name': 'Parasect', 'supertype'...     1        10     10   
7  {'id': 'g1-8', 'name': 'Tangela', 'supertype':...     1        10     10   
8  {'id': 'g1-9', 'name': 'Pinsir', 'supertype': ...     1        10     10   
9  {'id': 'g1-10', 'name': 'Leafeon-EX', 'superty...     1        10     10   

   totalCount  
0         117  
1         117  
2         117  
3         117  
4         117  
5         117  
6         117  
7 

In [ ]:
import json
import psycopg2
from psycopg2.extras import execute_values
import os
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv

from pokemon_tcg_service import PokemonTCGServer

# Set up database connection
load_dotenv()


DB_CONFIG = {
    "host": "postgres",
    "port": 5432,
    "database": os.getenv("POSTGRES_DB"),
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD")
}

# SQL statements for raw cards table
create_sql = """
    CREATE TABLE IF NOT EXISTS raw_cards (
    data JSONB,
    page INTEGER,
    pageSize INTEGER,
    count INTEGER,
    totalCount INTEGER
);
"""

insert_sql = """
    INSERT INTO raw_cards (data, page, pageSize, count, totalCount)
    VALUES %s;
"""


def create_raw_table():
    """Creates the raw_cards table for unprocessed card data"""
    try:
        # Establish connection to the database
        with psycopg2.connect(**DB_CONFIG) as conn:
            # Create a cursor object to execute SQL commands
            with conn.cursor() as cursor:
                cursor.execute(create_sql)
    except Exception as e:
        print(f"Error creating raw_cards table: {e}")

def fetch_cards_data():
    """Fetches cards data from the API"""
    server = PokemonTCGServer()
    response = server.get_cards(page_size=10, order_by="set.releaseDate", query="set.name:generations")
    data = response.json()
    
    cards = data.get("data", [])
    page_info = {
        "page": data.get("page"),
        "pageSize": data.get("pageSize"),
        "count": data.get("count"),
        "totalCount": data.get("totalCount")
    }
    return cards, page_info
    return cards

def load_to_db(cards: List[Dict[str, Any]], page_info: Dict[str, Any]):
    """Inserts cards into the raw_cards table"""
    rows = [
            (json.dumps(card), 
             page_info["page"], 
             page_info["pageSize"], 
             page_info["count"], 
             page_info["totalCount"])
            for card in cards
        ]
    try:

        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cursor:
                execute_values(cursor, insert_sql, rows)
    except Exception as e:
        print(f"Error inserting cards into raw_cards table: {e}")


create_raw_table()
cards = fetch_cards_data()
load_to_db(cards)